In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Classical feedforward and control flow (dynamische Schaltungen)

import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';



# Classical Feedforward und Kontrollfluss
{/*
  DO NOT EDIT THIS CELL!!!
  This cell's content is generated automatically by a script. Anything you add
  here will be removed next time the notebook is run. To add new content, create
  a new cell before or after this one.
*/}

<details>
<summary><b>Paketversionen</b></summary>

Der Code auf dieser Seite wurde mit den folgenden Anforderungen entwickelt.
Wir empfehlen die Verwendung dieser oder neuerer Versionen.

```
qiskit[all]~=2.3.0
qiskit-ibm-runtime~=0.43.1
```
</details>
> **Note:** Die neue Version dynamischer Schaltungen ist jetzt für alle Benutzer auf allen Backends verfügbar. Sie können dynamische Schaltungen nun im Utility-Maßstab ausführen. Weitere Einzelheiten finden Sie in [der Ankündigung](/announcements/product-updates/2025-09-25-new-dynamic-circuits).

Dynamische Schaltungen sind leistungsstarke Werkzeuge, mit denen Sie Qubits während der Ausführung einer Quantenschaltung messen und anschließend klassische Logikoperationen innerhalb der Schaltung basierend auf dem Ergebnis dieser Mid-Circuit-Messungen durchführen können. Dieser Prozess wird auch als _Classical Feedforward_ bezeichnet. Obwohl es noch frühe Tage sind, um zu verstehen, wie man dynamische Schaltungen am besten nutzt, hat die Quantenforschungsgemeinschaft bereits eine Reihe von Anwendungsfällen identifiziert, wie zum Beispiel:

* Effiziente Vorbereitung von Quantenzuständen, wie [GHZ-Zustand,](https://journals.aps.org/prxquantum/abstract/10.1103/PRXQuantum.5.030339) [W-Zustand,](https://arxiv.org/abs/2403.07604) (weitere Informationen zum W-Zustand finden Sie auch unter ["State preparation by shallow circuits using feed forward"](https://arxiv.org/abs/2307.14840)) und eine breite Klasse von [Matrix-Produktzuständen](https://arxiv.org/abs/2404.16083)
* [Effiziente Langstrecken-Verschränkung](https://journals.aps.org/prxquantum/abstract/10.1103/PRXQuantum.5.030339) zwischen Qubits auf demselben Chip durch Verwendung flacher Schaltungen
* Effizientes [Sampling von IQP-ähnlichen Schaltungen](https://arxiv.org/pdf/2505.04705)

Diese Verbesserungen durch dynamische Schaltungen bringen jedoch Kompromisse mit sich. Mid-Circuit-Messungen und klassische Operationen haben typischerweise längere Ausführungszeiten als Zwei-Qubit-Gates, und diese Zeitverlängerung könnte die Vorteile reduzierter Schaltungstiefe zunichte machen. Daher ist die Verkürzung der Mid-Circuit-Messungszeit ein Schwerpunktbereich für Verbesserungen, während IBM Quantum&reg; die [neue Version](/announcements/product-updates/2025-03-03-new-version-dynamic-circuits) dynamischer Schaltungen veröffentlicht.

Die [OpenQASM 3-Spezifikation](https://openqasm.com/language/classical.html#looping-and-branching) definiert eine Reihe von Kontrollflussstrukturen, aber Qiskit Runtime unterstützt derzeit nur die bedingte `if`-Anweisung. In Qiskit SDK entspricht dies der [if_test](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#if_test)-Methode auf [QuantumCircuit.](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit) Diese Methode gibt einen [Context Manager](https://docs.python.org/3/reference/datamodel.html#with-statement-context-managers) zurück und wird typischerweise in einer `with`-Anweisung verwendet. Dieser Leitfaden beschreibt, wie Sie diese bedingte Anweisung verwenden.

> **Note:** Die Code-Beispiele in diesem Leitfaden verwenden die Standard-Messanweisung für Mid-Circuit-Messungen. Es wird jedoch empfohlen, stattdessen die [`MidCircuitMeasure`](/guides/measure-qubits#midcircuit)-Anweisung zu verwenden, falls das Backend dies unterstützt. Siehe die [Mid-Circuit-Messungen-Dokumentation](/guides/measure-qubits#mid-circuit-measurements) für Details.
## `if`-Anweisung
Die `if`-Anweisung wird verwendet, um Operationen basierend auf dem Wert eines klassischen Bits oder Registers bedingt auszuführen.

Im folgenden Beispiel wenden wir ein Hadamard-Gate auf ein Qubit an und messen es. Wenn das Ergebnis 1 ist, wenden wir ein X-Gate auf das Qubit an, was den Effekt hat, es zurück in den 0-Zustand zu kippen. Wir messen dann das Qubit erneut. Das resultierende Messergebnis sollte mit 100%iger Wahrscheinlichkeit 0 sein.

In [1]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister

qubits = QuantumRegister(1)
clbits = ClassicalRegister(1)
circuit = QuantumCircuit(qubits, clbits)
(q0,) = qubits
(c0,) = clbits

circuit.h(q0)
# Use MidCircuitMeasure() if it's supported by the backend.
# circuit.append(MidCircuitMeasure(), [q0], [c0])
circuit.measure(q0, c0)
with circuit.if_test((c0, 1)):
    circuit.x(q0)
circuit.measure(q0, c0)
circuit.draw("mpl")

# example output counts: {'0': 1024}

<Image src="../docs/images/guides/classical-feedforward-and-control-flow/extracted-outputs/60924bfa-50ed-4d9d-a17b-9d64f2cc053f-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/classical-feedforward-and-control-flow/extracted-outputs/60924bfa-50ed-4d9d-a17b-9d64f2cc053f-0.svg)

Der `with`-Anweisung kann ein Zuweisungsziel gegeben werden, das selbst ein Context Manager ist, der gespeichert und anschließend zur Erstellung eines else-Blocks verwendet werden kann, der ausgeführt wird, wenn die Inhalte des `if`-Blocks *nicht* ausgeführt werden.

Im folgenden Beispiel initialisieren wir Register mit zwei Qubits und zwei klassischen Bits. Wir wenden ein Hadamard-Gate auf das erste Qubit an und messen es. Wenn das Ergebnis 1 ist, wenden wir ein Hadamard-Gate auf das zweite Qubit an; andernfalls wenden wir ein X-Gate auf das zweite Qubit an. Schließlich messen wir auch das zweite Qubit.

In [2]:
qubits = QuantumRegister(2)
clbits = ClassicalRegister(2)
circuit = QuantumCircuit(qubits, clbits)
(q0, q1) = qubits
(c0, c1) = clbits

circuit.h(q0)
circuit.measure(q0, c0)
with circuit.if_test((c0, 1)) as else_:
    circuit.h(q1)
with else_:
    circuit.x(q1)
circuit.measure(q1, c1)

circuit.draw("mpl")

# example output counts: {'01': 260, '11': 272, '10': 492}

<Image src="../docs/images/guides/classical-feedforward-and-control-flow/extracted-outputs/20f0640a-a3f7-41b3-aada-b66bc89b0555-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/classical-feedforward-and-control-flow/extracted-outputs/20f0640a-a3f7-41b3-aada-b66bc89b0555-0.svg)

Neben der Bedingung auf einem einzelnen klassischen Bit ist es auch möglich, auf den Wert eines klassischen Registers zu bedingen, das aus mehreren Bits besteht.

Im folgenden Beispiel wenden wir Hadamard-Gates auf zwei Qubits an und messen sie. Wenn das Ergebnis `01` ist, das heißt, das erste Qubit ist 1 und das zweite Qubit ist 0, wenden wir ein X-Gate auf ein drittes Qubit an. Schließlich messen wir das dritte Qubit. Beachten Sie, dass wir aus Gründen der Klarheit den Zustand des dritten klassischen Bits, der 0 ist, in der `if`-Bedingung angegeben haben. In der Schaltungszeichnung wird die Bedingung durch die Kreise auf den klassischen Bits angezeigt, auf die bedingt wird. Ein schwarzer Kreis zeigt eine Bedingung auf 1 an, während ein weißer Kreis eine Bedingung auf 0 anzeigt.

In [3]:
qubits = QuantumRegister(3)
clbits = ClassicalRegister(3)
circuit = QuantumCircuit(qubits, clbits)
(q0, q1, q2) = qubits
(c0, c1, c2) = clbits

circuit.h([q0, q1])
circuit.measure(q0, c0)
circuit.measure(q1, c1)
with circuit.if_test((clbits, 0b001)):
    circuit.x(q2)
circuit.measure(q2, c2)

circuit.draw("mpl")

# example output counts: {'101': 269, '011': 260, '000': 252, '010': 243}

<Image src="../docs/images/guides/classical-feedforward-and-control-flow/extracted-outputs/98e8f552-4169-42a3-8182-e14e9ffb59e2-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/classical-feedforward-and-control-flow/extracted-outputs/98e8f552-4169-42a3-8182-e14e9ffb59e2-0.svg)

## Klassische Ausdrücke
Das Qiskit-Modul für klassische Ausdrücke [`qiskit.circuit.classical`](https://docs.quantum.ibm.com/api/qiskit/circuit_classical) enthält eine experimentelle Darstellung von Laufzeitoperationen auf klassischen Werten während der Schaltungsausführung. Aufgrund von Hardwareeinschränkungen werden derzeit nur `QuantumCircuit.if_test()`-Bedingungen unterstützt.

Das folgende Beispiel zeigt, dass Sie die Berechnung der Parität verwenden können, um einen n-Qubit-GHZ-Zustand mit dynamischen Schaltungen zu erstellen. Erzeugen Sie zunächst $n/2$ Bell-Paare auf benachbarten Qubits. Kleben Sie dann diese Paare zusammen, indem Sie eine Schicht von CNOT-Gates zwischen den Paaren verwenden. Messen Sie dann das Ziel-Qubit aller vorherigen CNOT-Gates und setzen Sie jedes gemessene Qubit auf den Zustand $\vert 0 \rangle$ zurück. Wenden Sie $X$ auf jede ungemessene Stelle an, für die die Parität aller vorhergehenden Bits ungerade ist. Schließlich werden CNOT-Gates auf die gemessenen Qubits angewendet, um die bei der Messung verlorene Verschränkung wiederherzustellen.

In der Paritätsberechnung beinhaltet das erste Element des konstruierten Ausdrucks das Anheben des Python-Objekts `mr[0]` zu einem [`Value`](https://docs.quantum.ibm.com/api/qiskit/circuit_classical#value)-Knoten (`lift` wird verwendet, um beliebige Objekte in klassische Ausdrücke umzuwandeln). Dies ist für `mr[1]` und das mögliche folgende klassische Register nicht notwendig, da sie Eingaben für `expr.bit_xor` sind und das notwendige Anheben in diesen Fällen automatisch erfolgt. Solche Ausdrücke können in Schleifen und anderen Konstrukten aufgebaut werden.

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.classical import expr

num_qubits = 8
if num_qubits % 2 or num_qubits < 4:
    raise ValueError("num_qubits must be an even integer ≥ 4")
meas_qubits = list(range(2, num_qubits, 2))  # qubits to measure and reset

qr = QuantumRegister(num_qubits, "qr")
mr = ClassicalRegister(len(meas_qubits), "m")
qc = QuantumCircuit(qr, mr)

# Create local Bell pairs
qc.reset(qr)
qc.h(qr[::2])
for ctrl in range(0, num_qubits, 2):
    qc.cx(qr[ctrl], qr[ctrl + 1])

# Glue neighboring pairs
for ctrl in range(1, num_qubits - 1, 2):
    qc.cx(qr[ctrl], qr[ctrl + 1])

# Measure boundary qubits between pairs,reset to 0
for k, q in enumerate(meas_qubits):
    qc.measure(qr[q], mr[k])
    qc.reset(qr[q])

# Parity-conditioned X corrections
# Each non-measured qubit gets flipped iff the parity (XOR) of all
# preceding measurement bits is 1
for tgt in range(num_qubits):
    if tgt in meas_qubits:  # skip measured qubits
        continue
    # all measurement registers whose physical qubit index < tgt
    left_bits = [k for k, q in enumerate(meas_qubits) if q < tgt]
    if not left_bits:  # skip if list empty
        continue

    # build XOR-parity expression
    parity = expr.lift(
        mr[left_bits[0]]
    )  # lift the first bit to Value so it will be treated like a boolean.
    for k in left_bits[1:]:
        parity = expr.bit_xor(
            mr[k], parity
        )  # calculate parity with all other bits
    with qc.if_test(parity):  # Add X if parity is 1
        qc.x(qr[tgt])

# Re-entangle measured qubits
for ctrl in range(1, num_qubits - 1, 2):
    qc.cx(qr[ctrl], qr[ctrl + 1])

In [5]:
qc.draw(output="mpl", style="iqp", idle_wires=False, fold=-1)

<Image src="../docs/images/guides/classical-feedforward-and-control-flow/extracted-outputs/d0f0abdb-50d5-408d-a704-a1a555acdd85-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/classical-feedforward-and-control-flow/extracted-outputs/d0f0abdb-50d5-408d-a704-a1a555acdd85-0.svg)

## Backends finden, die dynamische Schaltungen unterstützen
Um alle Backends zu finden, auf die Ihr Konto zugreifen kann und die dynamische Schaltungen unterstützen, führen Sie Code wie den folgenden aus. Dieses Beispiel geht davon aus, dass Sie [Ihre Anmeldedaten gespeichert haben.](/guides/save-credentials) Sie könnten auch [explizit Anmeldedaten angeben](/guides/initialize-account#explicit) beim Initialisieren Ihres Qiskit Runtime-Dienstkontos. Dies würde es Ihnen ermöglichen, beispielsweise Backends anzuzeigen, die für eine bestimmte Instanz oder einen Plantyp verfügbar sind.

> **Note:** - Die Backends, die für das Konto verfügbar sind, hängen von der in den Anmeldedaten angegebenen Instanz ab.
> - Die neue Version dynamischer Schaltungen ist jetzt für alle Benutzer auf allen Backends verfügbar. Weitere Einzelheiten finden Sie in [der Ankündigung](/announcements/product-updates/2025-09-25-new-dynamic-circuits).

In [6]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()
dc_backends = service.backends(dynamic_circuits=True)
print(dc_backends)

[<IBMBackend('ibm_pittsburgh')>, <IBMBackend('ibm_boston')>, <IBMBackend('ibm_fez')>, <IBMBackend('ibm_miami')>, <IBMBackend('ibm_marrakesh')>, <IBMBackend('ibm_torino')>, <IBMBackend('ibm_kingston')>]


## Qiskit Runtime limitations

Be aware of the following constraints when running dynamic circuits in Qiskit Runtime.

- Due to the limited physical memory on control electronics, there is also a limit on the number of `if` statements and size of their operands. This limit is a function of the number of broadcasts and the number of broadcasted bits in a job (not a circuit).

   When processing an `if` condition, measurement data needs to be transferred to the control logic to make that evaluation. A broadcast is a transfer of unique classical data, and broadcasted bits is the number of classical bits being transferred. Consider the following:

   ```python
   c0 = ClassicalRegister(3)
   c1 = ClassicalRegister(5)
   ...
   with circuit.if_test((c0, 1)) ...
   with circuit.if_test((c0, 3)) ...
   with circuit.if_test((c1[2], 1)) ...
   ```
   In the previous code example, the first two `if_test` objects on `c0` are considered one broadcast because the content of `c0` did not change, and thus does not need to be re-broadcasted. The `if_test` on `c1` is a second broadcast. The first one broadcasts all three bits in `c0` and the second broadcasts just one bit, making a total of four broadcasted bits.

   Currently, if you broadcast 60 bits each time, then the job can have approximately 300 broadcasts. If you broadcast just one bit each time, however, then the job can have 2400 broadcasts.

- The operand used in an `if_test` statement must be 32 or fewer bits. Thus, if you are comparing an entire `ClassicalRegister`, the size of that `ClassicalRegister` must be 32 or fewer bits. If you are comparing just a single bit from a `ClassicalRegister`, however, that `ClassicalRegister` can be of any size (since the operand is only one bit).

   For example, the "Not valid" code block does not work because `cr` is more than 32 bits.  You can, however, use a classical register wider than 32 bits if you are testing only one bit, as shown in the "Valid" code block.

   <Tabs>
   <TabItem value="Not valid" label="Not valid">
      ```python
         cr = ClassicalRegister(50)
         qr = QuantumRegister(50)
         circuit = QuantumCircuit(qr, cr)
         ...
         circ.measure(qr, cr)
         with circ.if_test((cr, 15)):
            ...
      ```
   </TabItem>
   <TabItem value="Valid" label="Valid">
      ```python
         cr = ClassicalRegister(50)
         qr = QuantumRegister(50)
         circuit = QuantumCircuit(qr, cr)
         ...
         circ.measure(qr, cr)
         with circ.if_test((cr[5], 1)):
            ...
      ```
   </TabItem>
   </Tabs>

- Nested conditionals are not allowed. For example, the following code block will not work because it has an `if_test` inside another `if_test`:
   <Tabs>
    <TabItem value="Not valid" label="Not valid">
        ```python
           c1 = ClassicalRegister(1, "c1")
           c2 = ClassicalRegister(2, "c2")
           ...
           with circ.if_test((c1, 1)):
            with circ.if_test(c2, 1)):
             ...
        ```
     </TabItem>
     <TabItem value="Valid" label="Valid">
        ```python
        cr = ClassicalRegister(2)
        ...
        with circuit.if_test((cr, 0b11)):
          ...
        ```
     </TabItem>
    </Tabs>

- Having `reset` or measurements inside conditionals is not supported.
- Arithmetic operations are not supported.
- See the [OpenQASM 3 feature table](/docs/guides/qasm-feature-table) to determine which OpenQASM 3 features are supported on Qiskit and Qiskit Runtime.
- When OpenQASM 3 (instead of `QuantumCircuit`) is used as the input format to pass circuits to Qiskit Runtime primitives, only instructions that can be loaded into Qiskit are supported. Classical operations, for example, are not supported because they cannot be loaded into Qiskit. See [Import an OpenQASM 3 program into Qiskit](/docs/guides/interoperate-qiskit-qasm3#import-an-openqasm-3-program-into-qiskit) for more information.
- The `for`, `while`, and `switch` instructions are not supported.

## Use dynamic circuits with Estimator

Since Estimator does not support dynamic circuits, you can use Sampler and build your own measurement circuits instead. Alternatively, you can use the [Executor primitive,](/docs/guides/directed-execution-model#executor-primitive) which supports dynamic circuits.

To replicate Estimator's behavior, follow this process:

1. Group the terms of all observables into a partition.  This can be done by using the [`PauliList` API,](/docs/api/qiskit/qiskit.quantum_info.PauliList#group_qubit_wise_commuting) for example.
     <Admonition type="note">
      You can use the [`BitArray`](https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.primitives.BitArray#expectation_values) primitive attribute to compute the expectation values of the provided observables.
     </Admonition>
2. Execute one basis change circuit per partition (whichever basis change needs to be done for each partition). See the Measurement bases addon utility  [`measurement_bases` module](https://github.com/Qiskit/qiskit-addon-utils/blob/38ea05431f2e9830adf4ec9265f6d19758a32096/qiskit_addon_utils/exp_vals/measurement_bases.py) for more information. [Get started with utilities.](/docs/guides/qiskit-addons-utils#get-started-with-utilities)
3. Add back together the results for each partition.

## Next steps

<Admonition type="tip" title="Recommendations">
- Learn how to implement accurate dynamic decoupling by using [stretch.](/docs/guides/stretch)
- Learn about the shorter [mid-circuit measurements](/docs/guides/measure-qubits#mid-circuit-measurements) that reduce the circuit time.
- Use [circuit schedule visualization](/docs/guides/visualize-circuit-timing#qiskit-runtime-support) to debug and optimize your dynamic circuits.
</Admonition>